In [2]:
import rebound
import numpy as np

def calculate_vx(a):
    sim = rebound.Simulation()
    sim.add(m=1.) # star
    # for variation, need to define star as 'primary' explicitly ie give it position 0 in the particles array
    sim.add(primary=sim.particles[0],m=0.0009543, a=1) # A jupiter mass planet at a distance of 1 au
    sim.add(primary=sim.particles[0],m=0.00009543,a=a) # a 1/10 jupiter mass planet at a initial distance of a au
    sim.integrate(2.*np.pi*10) # integrate sim for 10 years
    return sim.particles[0].vx  # return star's velocity in the x direction

print calculate_vx(a=1.5)
print calculate_vx(a=1.5001)

7.10473913102e-05
7.12884724484e-05


In [3]:
def calculate_vx_derivative(a):
    sim = rebound.Simulation()
    sim.add(m=1.) # star
    sim.add(primary=sim.particles[0],m=0.0009543, a=1) # A jupiter mass planet at a distance of 1 au
    sim.add(primary=sim.particles[0],m=0.00009543,a=a) # a 1/10 jupiter mass planet at a initial distance of a au
    v1 = sim.add_variation()    # add a set of variational particles
    v1.vary(2,"a")              # initialize the variational particles --> the 2 is the particle index, 
                                #                                           and 'a' the parameter being varied

    sim.integrate(2.*np.pi*10.) # integrate for ~10 orbits
    return sim.particles[0].vx, v1.particles[0].vx # return star's velocity and its derivative

vel_0, dvel_0 = calculate_vx_derivative(a=1.5) # this will give the velocity at a=1.5, and the derivative
# Can then use a taylor expansion to get velocities for small perturbations to a=1.5
# Recall the taylor expansion in general, for f(x)~f(a)+ (x-a)df/dx + 0.5*(x-a)^2*(d^2f/dx)+...
# ---->  The expansion of v(a) will be v(a) = v(a0) + (a-a0)dv/dt, find this as an estimate for vx for a=1.5+e where e is 
#          small
a0 = 1.5
def find_v(a):
    return vel_0 + (a-a0)*dvel_0

print find_v(1.5001)


7.12885725463e-05


In [16]:
'''
So now let me try to vary the x position.
Lets vary the x position of Mars slightly, and see the impact it has on Earth's x position

--> will need to gind cartesian coords in terms of orbital parameters
'''

def calculate_xpos(x):
    sim = rebound.Simulation()
    sim.add(m=1)   # add the Sun
    ''' CANNOT USE CARTESIAN DATA, NEED TO USE ORBITAL ELEMENTS
    # add Earth as of Feb 13 2017, 00:00:00
    sim.add(m=2.95e-6, x=-7.978572823262211E-01, y= 5.812732071419280E-01, z=-1.722649673541333E-04,\
            vx=-1.033661818510841E-02, vy=-1.401082240323706E-02, vz=7.127882424861757E-10)   
    # add Mars as of Feb 13 2017, 00:00:00
    sim.add(m=3.21e-7, x=1.093396336138831, y=9.655017747507859E-01, z=-6.763596943826665E-03,\
            vx=-8.725401522523147E-03, vy=1.169825170487331E-02, vz=4.591341975755195E-04)  
    '''
    sim.add(primary=sim.particles[0], m=2.95e-6, a=9.991662909158031E-01, e= 1.620151388838157E-02, inc=1.102181520451029E-03,\
           Omega=2.446006429849909E+02, omega=2.160884075157936E+02, f=4.354571005004216E+01)
    # Mars a:  a=1.523704095698228E+00
    sim.add(primary=sim.particles[0], m=3.21e-7,a=x, e=9.350710936656478E-02, inc=1.848383565629012E+00,\
           Omega=4.950721528118110E+01, omega=2.866646515502128E+02, f=6.522977681625729E+01)
    
    sim.integrate(2*np.pi*1) # integrate for 1 Earth year
    return sim.particles[1].x # get Earth's x position back

# find Earth's x position at x=1.0 and a small change x=1.01
print "Earth's x position as a result of a change in Mars' semi major axis after 10 orbits "
print "At x=2.: {}, and at x=2.: {}".format(calculate_xpos(2.), calculate_xpos(2.5))

def calculate_xpos_deriv(x):
    sim = rebound.Simulation()
    sim.add(m=1)   # add the Sun
    sim.add(primary=sim.particles[0], m=2.95e-6, a=9.991662909158031E-01, e= 1.620151388838157E-02, inc=1.102181520451029E-03,\
           Omega=2.446006429849909E+02, omega=2.160884075157936E+02, f=4.354571005004216E+01)
    # Mars a:  a=1.523704095698228E+00
    sim.add(primary=sim.particles[0], m=3.21e-7,a=x, e=9.350710936656478E-02, inc=1.848383565629012E+00,\
           Omega=4.950721528118110E+01, omega=2.866646515502128E+02, f=6.522977681625729E+01)
    v1 = sim.add_variation() # add a set of variational particles
    v1.vary(2,"a")
    sim.integrate(2*np.pi*1) # integrate for 1 Earth year
    return sim.particles[1].x, v1.particles[1].x # get Earth's x position back, and its derivative
    
xpos, dxpos = calculate_xpos_deriv(2.)
x_0 = 2.
def find_x(x):
    return xpos + (x-x_0)*dxpos

print "At x=2. with variation: {}".format(find_x(2.5))

    

Earth's x position as a result of a change in Mars' semi major axis after 10 orbits 
At x=2.: -0.0170001127902, and at x=2.: -0.0169999096912
At x=2. with variation: -0.016999746299
